# Convert Terms for sDIC Decomposition

See Evernote journal from LANL summer, June 1st. Just conveting all the post-processed
means and residuals for each EBU for the terms relevant to the sDIC decomposition. This ensures that they are all mol/m3/yr.



In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def load_files(ebu, var):
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/' + var + '/' + ebu + 
               '/filtered_output/')
    ds1 = xr.open_dataset(filepath + ebu.lower() + '-' + var + 
                          '-forced-signal-AW-chavez-800km.nc')
    ds2 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-forced-signal-chavez-800km.nc')
    ds3 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-residuals-AW-chavez-800km.nc')
    ds4 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-residuals-chavez-800km.nc')
    return ds1, ds2, ds3, ds4

In [3]:
def convert_and_save(ds1, ds2, ds3, ds4, conversion, ebu, var):
    ds1 = ds1 * conversion
    ds1[var + '_AW'].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds1[var + '_AW'].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds1[var + '_AW'].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
    
    ds2 = ds2 * conversion
    ds2[var].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds2[var].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds2[var].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."

    ds3 = ds3 * conversion
    ds3[var + '_AW'].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds3[var + '_AW'].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds3[var + '_AW'].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
        
    ds4 = ds4 * conversion
    ds4[var].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds4[var].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds4[var].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
        
    outdir = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/' + 
              var + '/' + ebu + '/')
    # Save files
    ds1.to_netcdf(outdir + ebu.lower() + '-' + var + '-forced-signal-AW-chavez-800km.nc')
    ds2.to_netcdf(outdir + ebu.lower() + '-' + var + '-forced-signal-chavez-800km.nc')
    ds3.to_netcdf(outdir + ebu.lower() + '-' + var + '-residuals-AW-chavez-800km.nc')
    ds4.to_netcdf(outdir + ebu.lower() + '-' + var + '-residuals-chavez-800km.nc')

# Convert Terms

In [50]:
EBU = 'CalCS'
VAR = 'sDIC_int100m_tendency'

In [51]:
conversions = {'sDIC_int100m_tendency': (12/10**3),
               'Jint_100m_DIC': (60*60*24*365) / (10**5),
               'FG_CO2': 1}

In [ ]:
# Tendency conversion
ds1, ds2, ds3, ds4 = load_files(EBU, VAR)
convert_and_save(ds1, ds2, ds3, ds4, conversions[VAR], EBU, VAR)

# Make circulation residual

Ref: Lovenduski et al. 2007

$$
    \frac{d(sDIC^{\prime})}{dt} = J^{\prime}_{circ} + J^{\prime}_{ex} + J^{\prime}_{bio}
$$

Where $J^{\prime}_{ex}$ is positive in this case due to flipping the sign of FG_CO2. $ J^{\prime}_{bio}$ is also positive given the definition of it in CESM.

In [59]:
def load_updated_files(ebu, var):
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/' + var + '/' + ebu + '/')
    ds = xr.open_dataarray(filepath + ebu.lower() + '-' + var +
                          '-residuals-AW-chavez-800km.nc')
    return ds

In [93]:
EBU = 'BenCS'

In [94]:
co2_aw = load_updated_files(EBU, 'FG_CO2')
co2_aw = co2_aw.isel(ensemble=slice(1,34))

bio_aw = load_updated_files(EBU, 'Jint_100m_DIC')
bio_aw = bio_aw.isel(ensemble=slice(1,34))

tend_aw = load_updated_files(EBU, 'sDIC_int100m_tendency')

In [96]:
# Have to drop first ensemble member for consistency
circ_aw = tend_aw - co2_aw - bio_aw
circ_aw.name = 'Jint_100m_circ'
circ_aw.attrs['units'] = 'mol/m2/yr'
circ_aw.attrs['description'] = 'Residual computed by subtracting J_ex and J_bio from the reconstructed tendency term.'

####
outdir = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/Jint_100m_circ/' + EBU + '/')
outfile = (EBU.lower() + '-Jint_100m_circ-residuals-AW-chavez-800km.nc')
circ_aw.to_dataset().to_netcdf(outdir + outfile)